In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing, model_selection
from sklearn.metrics import log_loss
import string
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import stats
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.feature_extraction.text import TfidfTransformer
from keras.regularizers import l2, activity_l2
from sklearn.neighbors import KNeighborsClassifier
from keras.callbacks import Callback
import xgboost as xgb

class NBatchLogger(Callback):
    def __init__(self):
        self.seen = 0

    def on_epoch_end(self, batch, logs={}):
        self.seen += 1
        if self.seen % 500 == 0:
            print('epoch {}: loss = {}, val = {}'.format(self.seen, logs['loss'], logs['val_loss'])) 
            
class FakeDMatrix:
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels
        self.num = len(data)

    def num_row(self):
        return self.num

    def slice(self, rindex):
        indices = np.zeros(self.num, dtype=np.bool)
        for index in rindex:
            indices[index] = True
        return FakeDMatrix(data=self.data[indices], labels=self.labels[indices])

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [2]:
train_X = pd.read_json('data/train_with_leaky_cv.json')
train_y = pd.read_json('data/train_interest.json')[0]
columns = train_X.columns
train_X_scaled = pd.DataFrame(StandardScaler().fit_transform(train_X), columns=columns)
#train_X_with_layer_1 = pd.read_json('data/train_scaled_with_leaky_and_added_features_cv.json')

# Add a bunch of features to train_X

In [2]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.neighbors import DistanceMetric

In [4]:
list(train_X_scaled.columns)

['BoroCode',
 'Day',
 'Month',
 'NTACode',
 'Wday',
 'Yday',
 'Zero_building_id',
 'Zero_manager_id',
 'address_high_frac',
 'address_low_frac',
 'address_med_frac',
 'address_worths',
 'avenue',
 'bathrooms',
 'bc_price',
 'bedrooms',
 'boro_high_frac',
 'boro_low_frac',
 'boro_med_frac',
 'boro_worths',
 'building_high_frac',
 'building_id',
 'building_low_frac',
 'building_med_frac',
 'building_worths',
 'desc_letters_count',
 'desc_words_count',
 'desc_words_length',
 'display_address',
 'east',
 'features_count',
 'hour',
 'latitude',
 'longitude',
 'manager_high_frac',
 'manager_id',
 'manager_low_frac',
 'manager_med_frac',
 'manager_worths',
 'north',
 'nta_high_frac',
 'nta_low_frac',
 'nta_med_frac',
 'nta_worths',
 'other_address',
 'photos_count',
 'price_by_BoroCode',
 'price_by_NTACode',
 'price_by_area',
 'price_by_building_id',
 'price_by_display_address',
 'price_by_manager_id',
 'price_per_bathroom',
 'price_per_bed_and_bath',
 'price_per_bedroom',
 'real_minus_expect

In [7]:
ntrain = int(len(train_X) * 0.75)

scaler = StandardScaler()

xtr = train_X[:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X[ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]

In [16]:
NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

In [24]:
%%time
cols = [col for col in list(train_X.columns) if not col.endswith('worths')]
dtrain = xgb.DMatrix(data=xtr[cols], label=ytr)
dtest = xgb.DMatrix(data=xte[cols])

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])

bst = xgb.train(params, dtrain, best_rounds)
print('train score: ', log_loss(ytr_one_hot, bst.predict(dtrain)))
print('test score: ', log_loss(yte_one_hot, bst.predict(dtest)))
# 0.531

[0]	train-mlogloss:1.0915+1.37026e-05	test-mlogloss:1.09172+5.17301e-05
[200]	train-mlogloss:0.593105+0.00097267	test-mlogloss:0.625639+0.00312624
[400]	train-mlogloss:0.510146+0.00104157	test-mlogloss:0.566794+0.00367694
[600]	train-mlogloss:0.47098+0.000857232	test-mlogloss:0.550018+0.00341384
[800]	train-mlogloss:0.442976+0.000962218	test-mlogloss:0.542362+0.00310859
[1000]	train-mlogloss:0.42006+0.000760993	test-mlogloss:0.53793+0.00297711
[1200]	train-mlogloss:0.399763+0.000495187	test-mlogloss:0.535118+0.00287364
[1400]	train-mlogloss:0.381234+0.000621526	test-mlogloss:0.533206+0.0028097
[1600]	train-mlogloss:0.363973+0.000679936	test-mlogloss:0.531899+0.00255041
[1800]	train-mlogloss:0.347758+0.000647169	test-mlogloss:0.530938+0.00247888
[2000]	train-mlogloss:0.332424+0.000563903	test-mlogloss:0.530287+0.00243055
[2200]	train-mlogloss:0.318075+0.000614961	test-mlogloss:0.529914+0.00230364
0.529903
train score:  0.343920598363
test score:  0.537693293521
CPU times: user 3h 47min 

In [17]:
%%time
cols = [col for col in list(train_X.columns) if not col.startswith('sparse_')]
dtrain = xgb.DMatrix(data=xtr[cols], label=ytr)
dtest = xgb.DMatrix(data=xte[cols])

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])

bst = xgb.train(params, dtrain, best_rounds)
print('train score: ', log_loss(ytr_one_hot, bst.predict(dtrain)))
print('test score: ', log_loss(yte_one_hot, bst.predict(dtest)))

[0]	train-mlogloss:1.09144+1.57404e-05	test-mlogloss:1.09164+4.05788e-05
[200]	train-mlogloss:0.595084+0.0012245	test-mlogloss:0.627589+0.0036066
[400]	train-mlogloss:0.514481+0.00133087	test-mlogloss:0.57157+0.00447855
[600]	train-mlogloss:0.476159+0.00136191	test-mlogloss:0.556453+0.00456728
[800]	train-mlogloss:0.447741+0.00149133	test-mlogloss:0.549709+0.00430032
[1000]	train-mlogloss:0.424094+0.00160832	test-mlogloss:0.545748+0.00410376
[1200]	train-mlogloss:0.403252+0.00158531	test-mlogloss:0.543422+0.00385456
[1400]	train-mlogloss:0.384322+0.00152677	test-mlogloss:0.541796+0.0035792
[1600]	train-mlogloss:0.366528+0.00151377	test-mlogloss:0.540824+0.00342349
[1800]	train-mlogloss:0.349964+0.00154613	test-mlogloss:0.540202+0.00320404
[2000]	train-mlogloss:0.334427+0.0016487	test-mlogloss:0.539839+0.00306342
0.5397868
train score:  0.356810162847
test score:  0.548521112096
CPU times: user 1h 15min 49s, sys: 7min 28s, total: 1h 23min 17s
Wall time: 29min 50s


In [19]:
%%time
cols = [col for col in list(train_X.columns) if (not col.startswith('worth_')) and (not col.startswith('price_'))]
dtrain = xgb.DMatrix(data=xtr[cols], label=ytr)
dtest = xgb.DMatrix(data=xte[cols])

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])

bst = xgb.train(params, dtrain, best_rounds)
print('train score: ', log_loss(ytr_one_hot, bst.predict(dtrain)))
print('test score: ', log_loss(yte_one_hot, bst.predict(dtest)))

[0]	train-mlogloss:1.09156+1.58367e-05	test-mlogloss:1.09176+4.50671e-05
[200]	train-mlogloss:0.603364+0.00101354	test-mlogloss:0.63414+0.00377109
[400]	train-mlogloss:0.522066+0.00100241	test-mlogloss:0.576203+0.00448577
[600]	train-mlogloss:0.482695+0.000917929	test-mlogloss:0.558561+0.00427423
[800]	train-mlogloss:0.453882+0.00113939	test-mlogloss:0.5498+0.00405268
[1000]	train-mlogloss:0.430137+0.000853047	test-mlogloss:0.544584+0.00384638
[1200]	train-mlogloss:0.40918+0.000844477	test-mlogloss:0.541058+0.00357907
[1400]	train-mlogloss:0.39033+0.00069565	test-mlogloss:0.538678+0.00335179
[1600]	train-mlogloss:0.372895+0.000831401	test-mlogloss:0.536926+0.0033013
[1800]	train-mlogloss:0.356532+0.000868897	test-mlogloss:0.53575+0.00312374
[2000]	train-mlogloss:0.3412+0.00083446	test-mlogloss:0.534905+0.00309618
[2200]	train-mlogloss:0.326556+0.000780556	test-mlogloss:0.534452+0.00305813
[2400]	train-mlogloss:0.312724+0.000828834	test-mlogloss:0.5343+0.00295067
0.5343002
train score: 

In [8]:
%%time
params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

params = {
    'eta':.02,
    'colsample_bytree':.7,
    'subsample':.7,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1,
    
    'max_depth':4,
    'min_child_weight': 1,
}

cols = [col for col in list(train_X.columns) if col.startswith('sparse_')] + ['price_per_bedroom']
dtrain = xgb.DMatrix(data=xtr[cols], label=ytr)
dtest = xgb.DMatrix(data=xte[cols])

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])

bst = xgb.train(params, dtrain, best_rounds)
print('train score: ', log_loss(ytr_one_hot, bst.predict(dtrain)))
print('test score: ', log_loss(yte_one_hot, bst.predict(dtest)))

[0]	train-mlogloss:1.08821+5.87524e-05	test-mlogloss:1.08825+3.62844e-05
[200]	train-mlogloss:0.68701+0.000749359	test-mlogloss:0.694804+0.00382705
[400]	train-mlogloss:0.670175+0.000888663	test-mlogloss:0.683931+0.00381625
[600]	train-mlogloss:0.662602+0.000920479	test-mlogloss:0.680848+0.00360242
[800]	train-mlogloss:0.656657+0.000894507	test-mlogloss:0.679044+0.00347421
[1000]	train-mlogloss:0.651419+0.000845896	test-mlogloss:0.677794+0.00336274
[1200]	train-mlogloss:0.646637+0.000825468	test-mlogloss:0.676902+0.00327914
[1400]	train-mlogloss:0.642055+0.000789888	test-mlogloss:0.67624+0.0032678
[1600]	train-mlogloss:0.637828+0.000775101	test-mlogloss:0.67566+0.00328069
[1800]	train-mlogloss:0.63376+0.000789535	test-mlogloss:0.675215+0.00324592
[2000]	train-mlogloss:0.629968+0.000868979	test-mlogloss:0.67492+0.00324037
[2200]	train-mlogloss:0.626273+0.000786515	test-mlogloss:0.674645+0.00326982
[2400]	train-mlogloss:0.622771+0.000792332	test-mlogloss:0.674467+0.00333496
[2600]	train-

In [ ]:
[2600]	train-mlogloss:0.600032+0.000895735	test-mlogloss:0.671792+0.00350823
0.6717898
train score:  0.609815663253
test score:  0.670838713845
CPU times: user 3h 4min 57s, sys: 7min 30s, total: 3h 12min 28s
Wall time: 1h 1min 48s

In [17]:
params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

In [18]:
#%%time
layer_1_scaler = StandardScaler()
train_X_with_layer_1 = pd.read_json('data/train_with_leaky_cv.json')
train_y = pd.read_json('data/train_interest.json')[0]
columns_minus_layer_1 = train_X_with_layer_1.columns
train_X_scaled = pd.DataFrame(layer_1_scaler.fit_transform(train_X_with_layer_1), columns=columns_minus_layer_1)

n_components = 3
pca_columns = ['layer1_pca_{}'.format(i) for i in range(n_components)]
sparse_prediction_cols = ['predict_sparse_{}'.format(category) for category in ['high', 'medium', 'low']]
knn_n_closest = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
knn_columns = ['layer1_knn_{}_{}'.format(n_neighbors, category) for category in ['high', 'medium', 'low'] for n_neighbors in knn_n_closest]
ridge_columns = ['layer1_ridge_predictions']
rf_columns = ['layer1_rf_{}_{}_predictions'.format(criterion, category) for category in ['high', 'medium', 'low'] for criterion in ['gini', 'entropy']]
k_means_columns = ['layer1_k_means_{}_{}'.format(n_clusters, index) for n_clusters in [2, 4, 8] for index in range(n_clusters)]

all_layer_one_columns = pca_columns + sparse_prediction_cols + knn_columns + ridge_columns + rf_columns + k_means_columns
for column in all_layer_one_columns:
    train_X_with_layer_1[column] = 0

In [ ]:
all_layer_one_columns

['layer1_pca_0',
 'layer1_pca_1',
 'layer1_pca_2',
 'predict_sparse_high',
 'predict_sparse_medium',
 'predict_sparse_low',
 'layer1_knn_2_high',
 'layer1_knn_4_high',
 'layer1_knn_8_high',
 'layer1_knn_16_high',
 'layer1_knn_32_high',
 'layer1_knn_64_high',
 'layer1_knn_128_high',
 'layer1_knn_256_high',
 'layer1_knn_512_high',
 'layer1_knn_1024_high',
 'layer1_knn_2_medium',
 'layer1_knn_4_medium',
 'layer1_knn_8_medium',
 'layer1_knn_16_medium',
 'layer1_knn_32_medium',
 'layer1_knn_64_medium',
 'layer1_knn_128_medium',
 'layer1_knn_256_medium',
 'layer1_knn_512_medium',
 'layer1_knn_1024_medium',
 'layer1_knn_2_low',
 'layer1_knn_4_low',
 'layer1_knn_8_low',
 'layer1_knn_16_low',
 'layer1_knn_32_low',
 'layer1_knn_64_low',
 'layer1_knn_128_low',
 'layer1_knn_256_low',
 'layer1_knn_512_low',
 'layer1_knn_1024_low',
 'layer1_ridge_predictions',
 'layer1_rf_gini_high_predictions',
 'layer1_rf_entropy_high_predictions',
 'layer1_rf_gini_medium_predictions',
 'layer1_rf_entropy_medium_p

In [ ]:
#PCA
pca = PCA(n_components=n_components)
pca_representation = pca.fit_transform(train_X_scaled)
train_X_with_layer_1.loc[:, pca_columns] = pca_representation

#tsne = TSNE(n_components = 2)
#tsne_representation = tsne.fit_transform(train_X_scaled[knn_features[:15]][:5000])

folds = 5
full_length = len(train_X_scaled)
entries = int(full_length / folds)
full_index = train_X_scaled.index
fold_indices = []
fold_indices_keep = []
fold_indices_throw = []
for i in range(folds):
    print ('Fold {}'.format(i))
    indices = sorted(random.sample(list(full_index), entries))
    fold_indices.append(indices)
    full_index = full_index.drop(indices)

    full_index_keep = np.zeros((full_length), dtype=np.bool)
    full_index_throw = np.ones((full_length), dtype=np.bool)
    for j in fold_indices[i]:
        full_index_keep[j] = True
        full_index_throw[j] = False
    fold_indices_throw.append(full_index_throw)
    fold_indices_keep.append(full_index_keep)
    
    xtr = train_X_scaled[full_index_throw]
    ytr = train_y[full_index_throw]
    xte = train_X_scaled[full_index_keep]
    
    # SPARSE and PRICE
    cols = [col for col in list(train_X_scaled.columns) if col.startswith('sparse_')] + ['price_per_bedroom']
    dtrain = xgb.DMatrix(data=xtr[cols], label=ytr)
    dtest = xgb.DMatrix(data=xte[cols])
    bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
    best_rounds = np.argmin(bst['test-mlogloss-mean'])
    print (bst['test-mlogloss-mean'][best_rounds])
    bst = xgb.train(params, dtrain, best_rounds)
    train_X_with_layer_1.loc[indices, sparse_prediction_cols] = bst.predict(dtest)
    
    # KNN
    knn_features = ['bc_price', 'manager_worths', 'latitude', 'longitude', 'desc_words_length', 'Yday', 'building_worths', 'price_per_bedroom']
    for n_neighbors in knn_n_closest:
        #knn_dist = DistanceMetric.get_metric('mahalanobis', V=np.eye(len(knn_n_closest)))
        knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric='manhattan') # metric='manhattan'
        knn.fit(train_X_scaled[knn_features][full_index_throw], train_y[full_index_throw])
        knn_pred = knn.predict_proba(train_X_scaled[knn_features][full_index_keep])
        new_cols = ['layer1_knn_{}_{}'.format(n_neighbors, category) for category in ['high', 'medium', 'low']]
        train_X_with_layer_1.loc[indices, new_cols] = knn_pred
    
    ridge = Ridge()
    ridge.fit(xtr, ytr)
    ridge_predictions = ridge.predict(xte)
    train_X_with_layer_1.loc[indices, 'layer1_ridge_predictions'] = ridge_predictions
    
    for criterion in ['gini', 'entropy']:
        rf = RandomForestClassifier(
            n_estimators=25,
            criterion=criterion,
            max_features='auto',
            min_samples_split=20)

        rf.fit(xtr, ytr)
        rf_predictions = rf.predict_proba(xte)
        new_cols = ['layer1_rf_{}_{}_predictions'.format(criterion, category) for category in ['high', 'medium', 'low']]
        train_X_with_layer_1.loc[indices, new_cols] = rf_predictions
    
    for n_clusters in [2, 4, 8]:
        k_means = KMeans(n_clusters=n_clusters)
        k_means.fit(xtr[knn_features], ytr)
        k_means_indices = k_means.predict(xte[knn_features])
        new_cols = ['layer1_k_means_{}_{}'.format(n_clusters, index) for index in range(n_clusters)]
        train_X_with_layer_1.loc[indices, new_cols] = np.eye(n_clusters)[k_means_indices]

Fold 0
[0]	train-mlogloss:1.09334+1.46369e-05	test-mlogloss:1.09338+3.08765e-05
[200]	train-mlogloss:0.714642+0.00145697	test-mlogloss:0.724584+0.00286757
[400]	train-mlogloss:0.667322+0.00107876	test-mlogloss:0.685719+0.00439704
[600]	train-mlogloss:0.652947+0.000956845	test-mlogloss:0.677757+0.00493822
[800]	train-mlogloss:0.64474+0.000976871	test-mlogloss:0.674875+0.0051177
[1000]	train-mlogloss:0.638542+0.000992185	test-mlogloss:0.673386+0.00523967
[1200]	train-mlogloss:0.633118+0.000974065	test-mlogloss:0.672462+0.00532132
[1400]	train-mlogloss:0.62814+0.000962654	test-mlogloss:0.671779+0.00539465
[1600]	train-mlogloss:0.623502+0.000991192	test-mlogloss:0.671327+0.00542241
[1800]	train-mlogloss:0.618923+0.00100149	test-mlogloss:0.671017+0.00541125
[2000]	train-mlogloss:0.61449+0.00103191	test-mlogloss:0.670733+0.005445
[2200]	train-mlogloss:0.610249+0.00105572	test-mlogloss:0.67049+0.00548051


In [ ]:
train_X_with_layer_1.to_json('data/train_scaled_with_leaky_and_added_features_cv.json', orient='records')

In [276]:
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.layers.core import ActivityRegularization
def create_model(xtr, layer1=256, layer2=32, activation='tanh', activation2=PReLU, dropout=0.75):
    model = Sequential()
    
    model.add(Dense(layer1,
                    input_dim=xtr.shape[1],
                    init = 'he_normal',
                    activation=activation))
                    #kernel_regularizer=regularizers.l2(0.01),
                    #activity_regularizer=regularizers.activity_l2(0.005)))
    #model.add(Dense(64, input_dim=64, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
    
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    if activation2 != None:
        model.add(activation2())

    model.add(Dense(layer2,
                    init = 'he_normal',
                    activation=activation))
                    #activity_regularizer=regularizers.activity_l2(0.005)))
    model.add(BatchNormalization())    
    model.add(Dropout(dropout))
    if activation2 != None:
        model.add(activation2())

    model.add(Dense(3, init = 'he_normal', activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    return model

In [268]:
def test_nn_with_columns(X, y, columns_to_include, layer1, layer2, activation, activation2, dropout):
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    ntrain = int(len(X_scaled) * 0.75)

    xtr = X_scaled[columns_to_include][:ntrain]
    ytr = y[:ntrain].values
    ytr_one_hot = np.eye(3)[ytr]
    xte = X_scaled[columns_to_include][ntrain:]
    yte = y[ntrain:].values
    yte_one_hot = np.eye(3)[yte]

    filepath = 'data/nn_weights'

    logger = NBatchLogger()
    early_stop = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)
    model = create_model(xtr, layer1, layer2, activation, activation2, dropout)
    model.fit(xtr.values, ytr_one_hot, nb_epoch = 1200, batch_size=1000, verbose = 0, validation_data=[xte.values, yte_one_hot], callbacks=[logger, early_stop, checkpoint])
    model = create_model(xtr, layer1, layer2, activation, activation2, dropout)
    model.load_weights(filepath)
    return log_loss(yte_one_hot, model.predict(xte.values))

# actually build model

In [274]:
ntrain = int(len(train_X_with_layer_1) * 0.75)

scaler = StandardScaler()

xtr = train_X_with_layer_1[:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X_with_layer_1[ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]

In [317]:
%%time
NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

xgb_cols = all_columns + k_means_columns + knn_columns + ridge_columns + pca_columns + rf_columns

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

dtrain = xgb.DMatrix(data=xtr[xgb_cols], label=ytr)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

dtest = xgb.DMatrix(data=xte[xgb_cols])
bst = xgb.train(params, dtrain, best_rounds)
xgb_preds = bst.predict(dtest)
print(log_loss(yte_one_hot, xgb_preds))
# 0.539725

[0]	train-mlogloss:1.09111+1.37637e-05	test-mlogloss:1.09132+3.47597e-05
[200]	train-mlogloss:0.572365+0.000966518	test-mlogloss:0.606583+0.00289119
[400]	train-mlogloss:0.495784+0.00111757	test-mlogloss:0.556851+0.00274886
[600]	train-mlogloss:0.46142+0.00115805	test-mlogloss:0.546969+0.00247487
[800]	train-mlogloss:0.435288+0.00122307	test-mlogloss:0.543303+0.00232122
[1000]	train-mlogloss:0.412388+0.00110456	test-mlogloss:0.541574+0.00211891
[1200]	train-mlogloss:0.391678+0.0010726	test-mlogloss:0.540466+0.00189828
[1400]	train-mlogloss:0.372348+0.00112275	test-mlogloss:0.539871+0.00179687
0.5397984
1429
0.546284733463
CPU times: user 3h 44s, sys: 5min 4s, total: 3h 5min 48s
Wall time: 53min 48s


In [307]:
%%time
filepath = 'data/nn_weights'

nn_cols = all_columns + knn_columns + ridge_columns + k_means_columns

logger = NBatchLogger()
early_stop = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)
model = create_model(xtr[nn_cols])
model.fit(xtr[nn_cols].values, ytr_one_hot, nb_epoch = 1200, batch_size=1000, verbose = 0, validation_data=[xte[nn_cols].values, yte_one_hot], callbacks=[logger, early_stop, checkpoint])

model = create_model(xtr[nn_cols])
model.load_weights(filepath)
nn_preds = model.predict(xte[nn_cols].values)
print(log_loss(yte_one_hot, nn_preds))
# 0.58044096863

0.58044096863
CPU times: user 11min 51s, sys: 58.1 s, total: 12min 50s
Wall time: 10min 9s


In [310]:
%%time
et_cols = all_columns# + knn_columns + ridge_columns + k_means_columns

test = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=50)

test.fit(xtr[et_cols], ytr)
            
et_train_predictions = test.predict_proba(xtr[et_cols])
et_predictions = test.predict_proba(xte[et_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_predictions), log_loss(ytr_one_hot, et_train_predictions)))
# 0.5989926646715679

0.6084469079023 (test loss 0.5014715811903182)
CPU times: user 10 s, sys: 158 ms, total: 10.2 s
Wall time: 10.2 s


In [309]:
%%time
rf_cols = all_columns# + knn_columns + ridge_columns + k_means_columns

test = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=50)

test.fit(xtr[rf_cols], ytr)
            
rf_train_predictions = test.predict_proba(xtr[rf_cols])
rf_predictions = test.predict_proba(xte[rf_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_predictions), log_loss(ytr_one_hot, rf_train_predictions)))
# 0.5814357939419752

0.5867972591837729 (test loss 0.4744615099148531)
CPU times: user 11.2 s, sys: 297 ms, total: 11.5 s
Wall time: 11.6 s


In [304]:
%%time
test = KNeighborsClassifier(n_neighbors=500)
test.fit(xtr[non_sparse_cols], ytr)

knn_train_predictions = test.predict_proba(xtr[non_sparse_cols])
knn_predictions = test.predict_proba(xte[non_sparse_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))

0.6644185840766779 (test loss 0.6542689234451893)
CPU times: user 5min 38s, sys: 4.82 s, total: 5min 43s
Wall time: 5min 55s


In [305]:
%%time
test = KNeighborsClassifier(n_neighbors=500)
test.fit(xtr[knn_features], ytr)

knn_train_predictions = test.predict_proba(xtr[knn_features])
knn_predictions = test.predict_proba(xte[knn_features])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))

0.6542388412538083 (test loss 0.6351798773827041)
CPU times: user 49.6 s, sys: 1.34 s, total: 51 s
Wall time: 52.1 s


In [ ]:
print(log_loss(yte_one_hot, xgb_preds))
mixed_preds = (0.75 * xgb_preds) + (0.25 * nn_preds)
print(log_loss(yte_one_hot, mixed_preds))

In [ ]:
train thing on whether or not each classifier did a good job (minimize distance from correct?)
use result to go with the best prediction blended from models

In [45]:
#new_vects = ['tf_idf_' + x[7:] for x in sparse_cols]
#sparse = processed_train[sparse_cols]
#tf_idfs = TfidfTransformer().fit_transform(sparse)
#tf_idfs_df = pd.DataFrame(tf_idfs.toarray(), columns=new_vects)
#processed_train = processed_train.join(tf_idfs_df, how='left')

# Ridge Lasso Elastic

In [308]:
las = Lasso()
enet = ElasticNet()
las.fit(xte, yte)
enet.fit(xte, yte)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [309]:
print (las.score(xte, yte))
predictions = las.predict(xte)
print (log_loss(yte_one_hot, np.eye(3)[np.round(predictions).astype(int)]))

0.0
10.478087214


In [310]:
print (enet.score(xte, yte))
predictions = enet.predict(xte)
print (log_loss(yte_one_hot, np.eye(3)[np.round(predictions).astype(int)]))

0.0
10.478087214


In [42]:
for coef, column in zip(enet.coef_, train_X.columns):
    if coef != 0:
        print (column, ": ", coef)

In [43]:
for coef, column in zip(las.coef_, train_X.columns):
    if coef != 0:
        print (column, ": ", coef)

# TREES

In [322]:
test = ExtraTreesClassifier(
    n_estimators=100,
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
et_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_predictions), log_loss(ytr_one_hot, predictions)))

0.5843451108603969 (test loss 0.3912286072900344)


In [323]:
test = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
et_ent_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_ent_predictions), log_loss(ytr_one_hot, predictions)))

0.5841850290743527 (test loss 0.3945679429419886)


In [311]:
test = RandomForestClassifier(
    n_estimators=100,
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
_predictions = test.predict_proba(xtr)
_rf_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, _rf_predictions), log_loss(ytr_one_hot, _predictions)))

0.5702881670274706 (test loss 0.34260988988913094)


In [312]:
test = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
_predictions = test.predict_proba(xtr)
_rf_ent_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, _rf_ent_predictions), log_loss(ytr_one_hot, _predictions)))

0.5690015004443815 (test loss 0.34470505904504534)


In [337]:
pred_mixed = (0.01 * et_predictions + 0.01 * et_ent_predictions + 0.49 * rf_predictions + 0.49 * rf_ent_predictions)
log_loss(yte_one_hot, pred_mixed)

0.56688904952650898

In [12]:
%%time
test = KNeighborsClassifier(n_neighbors=2)
test.fit(xtr[non_sparse_cols], ytr)

predictions = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions = test.predict_proba(xte[non_sparse_cols])

7.076257047822155 (test loss 0.23129250627049086)
CPU times: user 3min 15s, sys: 1.48 s, total: 3min 16s
Wall time: 3min 20s


In [13]:
%%time
test = KNeighborsClassifier(n_neighbors=5)
test.fit(xtr[non_sparse_cols], ytr)

predictions_2 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_2 = test.predict_proba(xte[non_sparse_cols])

2.997161309463761 (test loss 0.4189513500335195)
CPU times: user 3min 36s, sys: 1.02 s, total: 3min 37s
Wall time: 3min 38s


In [14]:
%%time
test = KNeighborsClassifier(n_neighbors=10)
test.fit(xtr[non_sparse_cols], ytr)

predictions_3 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_3 = test.predict_proba(xte[non_sparse_cols])

1.5474359459916005 (test loss 0.4994428601539661)
CPU times: user 3min 48s, sys: 839 ms, total: 3min 49s
Wall time: 3min 50s


In [21]:
%%time
test = KNeighborsClassifier(n_neighbors=200)
test.fit(xtr[non_sparse_cols], ytr)

predictions_6 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_6 = test.predict_proba(xte[non_sparse_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_ent_predictions_6), log_loss(ytr_one_hot, predictions_6)))

0.6344317626130038 (test loss 0.6195293255389913)
CPU times: user 4min 34s, sys: 1.19 s, total: 4min 35s
Wall time: 4min 36s


In [ ]:
for feat, importance in zip(processed_train[cols].columns, clf.feature_importances_):
    if not np.isnan(importance) and importance > 0:
        print (feat, ": ", importance)

In [345]:
print ("nn: ", log_loss(yte_one_hot, nn_predictions))
print ("ab: ", log_loss(yte_one_hot, pred_mixed))
frac_nn = 0.2
averaged_preds = (nn_predictions * frac_nn) + (pred_mixed * (1 - frac_nn))
print ("combo: ", log_loss(yte_one_hot, averaged_preds))

nn:  0.563593691555
ab:  0.566889049527
combo:  0.55827286106


In [ ]:
train_X = pd.read_json('data/train_leaky.json')
train_y = pd.read_json('data/train_interest.json')[0]
scaler = StandardScaler().fit(train_X)
train_X = pd.DataFrame(scaler.transform(train_X), columns=columns)
test_X = pd.read_json('data/test_leaky.json')
test_X = pd.DataFrame(scaler.transform(test_X), columns=columns)
listing_id = pd.read_json('data/test_ids.json').values
preds = pd.read_csv('data/my_best_preds.csv')

In [295]:
preds = pd.read_csv('data/my_best_preds.csv')

In [313]:
et_clf = ExtraTreesClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='auto',
    min_samples_split=20)
et_clf.fit(train_X, train_y)          
et_predictions = et_clf.predict_proba(test_X)

In [314]:
et_clf_ent = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)
et_clf_ent.fit(train_X, train_y)          
et_ent_predictions = et_clf_ent.predict_proba(test_X)

In [316]:
rf_clf = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    min_samples_split=20)
rf_clf.fit(train_X, train_y)          
rf_predictions = rf_clf.predict_proba(test_X)

In [317]:
rf_clf_ent = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    min_samples_split=20)
rf_clf_ent.fit(train_X, train_y)          
rf_ent_predictions = rf_clf_ent.predict_proba(test_X)

In [259]:
nn_predictions = model.predict(test_X.values)

In [297]:
xgb_predictions = preds[['high', 'medium', 'low']].values

In [303]:
new_preds = (0.8 * xgb_predictions) + (0.15 * nn_predictions) + (0.05 * et_predictions)

In [304]:
new_preds_df = pd.DataFrame(new_preds, columns=['high', 'medium', 'low'])
new_preds_df['listing_id'] = preds['listing_id']

In [306]:
new_preds_df.to_csv('data/combined_preds.csv', index=None)